In [0]:
#first, upload the file to the databricks file system (DBFS). This is manual

# converting a geospatial file to a topojson.

## Why convert to a TopoJSON? ##
Most geojson files are coordinate based. A topoJSON uses a series of arcs to store its geometry. Why use TopoJSON? Because PowerBI's Shape Map visulaization component only works with TopoJSON files.

Note you could do this processing on your desktop computer as well, it might help you to save it locally.

First, upload the file to the databricks file system (DBFS). This is manual.
Second, start a cluster. We have clusters with geospatial tools installed and that would that would help you out. But serverless is faster and cheaper and you can use pip install to do this.

In [0]:
%pip install geopandas
%pip install topojson
%pip install geojson

In [0]:
dbutils.library.restartPython()

In [0]:
import geopandas as gpd
import topojson as tp
import geojson

In [0]:
filepath = "/Volumes/scorecard_fulcrum/scorecard_fulcrum_records/scorecard_fulcrum_volume/2020NTAMap.json"
filepath = "/Volumes/scorecard_fulcrum/geo/scoreacard_geospatial_files/community_districts_simplified.geojson"

#you can even do shapefiles.
filepath = "/Volumes/scorecard_fulcrum/geo/scoreacard_geospatial_files/BusinessImprovementDistrict.zip"
filepath = '/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/nypd.geojson'

In [0]:
#read the file. It can be a geojson, json, shapefile, any geopandas-compatible map.
gdf = gpd.read_file(filepath)
#gdf.info()
print(gdf['precinct'])

#you may need to simplify it if it is too large and you get a "not serializable error". The 0.0001 is the tolarance in degrees.
gdf['geometry'] = gdf.simplify(0.0001)
#gdf.info()
#convert GeoDataFrame to GeoJSON
my_geojson = gdf.to_json()

#convert GeoJSON to TopoJSON
my_topojson = tp.Topology(my_geojson)
print(my_topojson)

# Write the file to the volume #
From there you may download it to your local computer.

In [0]:
filename = filepath.split("/")[-1].split('.')[0]
directory = filepath.split("/")[:-1]
directory = "/".join(directory)
print(directory)
topojson_path = f"{directory}/{filename}_topojson.json"
with open(topojson_path, 'w') as f:
    f.write(my_topojson.to_json())


In [0]:
my_topojson